In [ ]:
import json
import boto3
import pymysql

# S3 및 RDS 설정
s3 = boto3.client('s3')
rds_host = "your_rds_endpoint"
username = "your_db_username"
password = "your_db_password"
database_name = "your_database_name"

# Lambda 핸들러 함수
def lambda_handler(event, context):
    # S3에서 데이터 가져오기
    response = s3.get_object(Bucket='your_bucket_name', Key='response_data.json')
    data = json.loads(response['Body'].read())
    
    # RDS에 연결
    connection = pymysql.connect(
        host=rds_host,
        user=username,
        password=password,
        db=database_name
    )
    
    with connection.cursor() as cursor:
        for entry in data:
            student_id = entry['student_id']
            session_id = entry['session_id']
            question_id = entry['question_id']
            response = entry['response']
            
            # RDS에서 해당 question_id의 문항모수를 가져와서 채점
            cursor.execute("SELECT a, b, c FROM question_bank WHERE question_id = %s", (question_id,))
            question_params = cursor.fetchone()
            
            if question_params:
                a, b, c = question_params
                # 예시: 3PL 모델을 사용하여 정답 확률 계산 및 채점
                probability_correct = c + (1 - c) / (1 + np.exp(-1.7 * a * (theta - b)))
                is_correct = int(response == 1 if probability_correct > 0.5 else 0)
                
                # SageMaker 호출하여 θ 값 업데이트 (생략: SageMaker 호출 코드 추가 필요)
                
                # RDS에 학생의 θ 값 업데이트
                cursor.execute("UPDATE student_abilities SET theta = %s WHERE student_id = %s", (new_theta, student_id))
        
        # 변경 사항 커밋
        connection.commit()

    connection.close()
    return {"statusCode": 200, "body": json.dumps("Processing completed successfully.")}
